# Naptha SDK: Getting Started

The **[Naptha Python SDK](https://pypi.org/project/naptha-sdk/)** provides a unified interface for building and running multi-agent systems across distributed nodes. This notebook will walk you through:

* [Installing and setting up](#setup) the Naptha SDK
* [Authentication and configuration](#authentication)
* [Exploring available nodes](#exploring-nodes)
* [Working with agents](#working-with-agents)
* [Managing tools](#managing-tools)
* [Using orchestrators](#using-orchestrators)
* [Working with knowledge bases](#working-with-knowledge-bases)
* [Managing memory](#managing-memory)
* [Using personas](#using-personas)
* [Local model inference](#local-model-inference)
* [Storage operations](#storage-operations)


## Setup

First, let's install the Naptha SDK:

In [ ]:
%pip install naptha-sdk

## Authentication

Before using the SDK, you need to authenticate. If you don't have an account, you can create one using the CLI. For a quick guide, check out the [Create a New User](https://docs.naptha.ai/Tutorials/create-a-new-user) documentation.

There are two ways to create an account:

**Method 1: Interactive Signup (Recommended)**
The simplest way is through the interactive CLI:

```bash
naptha signup
```
This will:
- Guide you through username/password creation
- Generate your cryptographic keypair
- Automatically save credentials to `.env`
 
**Method 2: Pre-configured Setup**
If you prefer setting credentials beforehand:
 
1. Edit your `.env` file:
```bash
HUB_USERNAME=your_username
HUB_PASSWORD=your_password
PRIVATE_KEY=your_private_key  # Optional - will be generated if not provided
```
 
2. Then run:
```bash
naptha signup
```
 
Once you have an account, set up your credentials:

In [ ]:
#now let's test our auth credentials
import os
from naptha_sdk.client.naptha import Naptha

# Option 1: Using environment variables
os.environ["HUB_USER"] = "your_username"
os.environ["HUB_PASS"] = "your_password"

# Option 2: Direct authentication
async def authenticate():
    async with Naptha() as naptha:
        await naptha.hub.signin("HUB_USER", "HUB_PASS") #await naptha.hub.signin(os.getenv("HUB_USER"), os.getenv("HUB_PASS"))
        return "Authentication successful!"

# Run authentication
import asyncio
print(asyncio.run(authenticate()))

Expected output:
```
Authentication successful!
```
The CLI typically returns the following upon successful authentication:
```
INFO - Attempting authentication for user: ****
INFO - Successfully authenticated user: ****
````

## Exploring Nodes

Let's see what nodes are available in the network:

In [ ]:
async def list_nodes():
    async with Naptha() as naptha:
        await naptha.hub.signin(os.getenv("HUB_USER"), os.getenv("HUB_PASS"))
        nodes = await naptha.hub.list_nodes()
        return nodes

nodes = asyncio.run(list_nodes())
print("Available Nodes:" )
# now let's print the nodes by extracting key details
for node in nodes:
    print(f"- ID: {node['id'][:15]}...")
    print(f"  IP: {node['ip']}")
    print(f"  OS: {node['os']}")
    print(f"  Architecture: {node['arch']}")
    print(f"  Models: {', '.join(node['models'])}")
    print(f"  Provider Types: {', '.join(node['provider_types'])}")
    print(f"  RAM: {node['ram'] / (1024**3):.2f} GB")
    print(f"  GPUs: {node['num_gpus']}")
    print()

Sample output for Available Nodes:
```
- ID: node:90875a428e...
  IP: node.naptha.ai
  OS: Linux
  Architecture: x86_64
  Models: phi3:mini
  Provider Types: models, storage, modules
  RAM: 15.61 GB
  GPUs: 0
```

## Working with Agents

### List Available Agents

In [ ]:
async def list_agents():
    async with Naptha() as naptha:
        await naptha.hub.signin(os.getenv("HUB_USER"), os.getenv("HUB_PASS"))
        agents = await naptha.hub.list_agents()
        return agents

agents = asyncio.run(list_agents())
print("Available Agents:")
for agent in agents:
    print(f"- Name: {agent['name']}")
    print(f"  Description: {agent['description']}")
    print(f"  Type: {agent['execution_type']}")
    print(f"  Version: {agent['module_version']}")
    print(f"  URL: {agent['module_url']}")
    print(f"  Parameters: {agent['parameters']}")
    print()

Sample output for available Agents:

- Name: random_number_agent
  Description: A simple agent for taking part in Keynesian beauty contests
  Type: package
  Version: v0.5
  URL: https://github.com/NapthaAI/random_number_agent
  Parameters: {agent_name: str}

### Run Simple Chat Agent

In [ ]:
async def run_simple_chat_agent(question):
    async with Naptha() as naptha:
        await naptha.hub.signin(os.getenv("HUB_USER"), os.getenv("HUB_PASS"))
        
        # Check/register user
        user = await naptha.node.check_user({"public_key": naptha.hub.public_key})
        if not user:
            user = await naptha.node.register_user({"public_key": naptha.hub.public_key})
        
        agent_run_input = {
            'consumer_id': user['id'],
            "inputs": {
                'tool_name': 'chat',
                'tool_input_data': question
            },
            "agent_deployment": {
                "name": "simple_chat_agent",
                "module": {
                    "name": "simple_chat_agent"
                },
                "worker_node_url": "http://node.naptha.ai:7001" #Naptha's hosted node
            }
        }
        
        return await naptha.node.run_agent_and_poll(agent_run_input)

# Example usage
result = asyncio.run(run_simple_chat_agent("What is artificial intelligence?"))
print("Agent Response:", result.results[0] if result.results else "No response")

## Managing Tools

### List Available Tools

In [ ]:
async def list_tools():
    async with Naptha() as naptha:
        await naptha.hub.signin(os.getenv("HUB_USER"), os.getenv("HUB_PASS"))
        tools = await naptha.hub.list_tools()
        return tools

tools = asyncio.run(list_tools())
print("Available Tools:")
for tool in tools:
    print(f"- Name: {tool['name']}")
    print(f"  Description: {tool['description']}")
    print(f"  Parameters: {tool['parameters']}")

## Managing Knowledge Bases

### List Available Knowledge Bases

In [ ]:
async def list_kbs():
    async with Naptha() as naptha:
        await naptha.hub.signin(os.getenv("HUB_USER"), os.getenv("HUB_PASS"))
        kbs = await naptha.hub.list_kbs(kb_name=kb_name) #e.g kb_name="wikipedia_kb"
        return kbs

kbs = asyncio.run(list_kbs())
print("Available Knowledge Bases:")
for kb in kbs:
    print(f"- Name: {kb['name']}")
    print(f"  Description: {kb['description']}")
    print(f"  Parameters: {kb['parameters']}")

## Managing Memory Modules

### List Available Memory Modules

In [ ]:
async def list_memories():
    async with Naptha() as naptha:
        await naptha.hub.signin(os.getenv("HUB_USER"), os.getenv("HUB_PASS"))
        memories = await naptha.hub.list_memories(memory_name=memory_name) #e.g memory_name="cognitive_memory"
        return memories

memories = asyncio.run(list_memories())
print("Available Memories:")
for memory in memories:
    print(f"- Name: {memory['name']}")
    print(f"  Description: {memory['description']}")
    print(f"  Parameters: {memory['parameters']}")

## Managing Personas

### List Available Persona Modules

In [ ]:
async def list_personas():
    async with Naptha() as naptha:
        await naptha.hub.signin(os.getenv("HUB_USER"), os.getenv("HUB_PASS"))
        personas = await naptha.hub.list_personas()
        return personas

personas = asyncio.run(list_personas())
print("Available Personas:")
for persona in personas:
    print(f"- Name: {persona['name']}")
    print(f"  Description: {persona['description']}")
    print(f"  Parameters: {persona['parameters']}")
        

## Managing Orchestrators

### List Available Orchestrators

In [ ]:
async def list_orchestrators():
    async with Naptha() as naptha:
        await naptha.hub.signin(os.getenv("HUB_USER"), os.getenv("HUB_PASS"))
        orchestrators = await naptha.hub.list_orchestrators()
        return orchestrators

orchestrators = asyncio.run(list_orchestrators())
print("Available Orchestrators:")
for orchestrator in orchestrators:
    print(f"- Name: {orchestrator['name']}")
    print(f"  Description: {orchestrator['description']}")
    print(f"  Parameters: {orchestrator['parameters']}")

## Local Model Inference

### Run Inference with Local Models on Naptha Nodes

In [ ]:
from naptha_sdk.schemas import ChatCompletionRequest

async def run_local_inference(prompt, model="phi3:mini"):
    async with Naptha() as naptha:
        await naptha.hub.signin(os.getenv("HUB_USER"), os.getenv("HUB_PASS"))
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
        req = ChatCompletionRequest(
            model=model,
            messages=messages
        )
        res = await naptha.node.run_inference(req)
        return res['choices'][0]['message']['content']

# Example usage
response = asyncio.run(run_local_inference("Explain the concept of multi-agent systems."))
print("Model Response:", response)

## Storage Operations

### Working with Different Storage Types
 
The Naptha Storage API provides a unified interface for working with different storage backends on Naptha Nodes:

- **Filesystem Storage**: For storing and managing files and directories
- **Database Storage**: For structured data storage with support for SQL databases
- **IPFS Storage**: For decentralized content-addressed storage

For example, let's create a table in a database storage provider.

In [ ]:
import asyncio
from naptha_sdk.schemas import NodeConfigUser
from naptha_sdk.storage.storage_provider import StorageProvider
from naptha_sdk.storage.schemas import CreateStorageRequest, StorageType

node = NodeConfigUser(ip="node.naptha.ai", http_port=7001, server_type="http")
storage_provider = StorageProvider(node)

schema = {
    "schema": {
        "id": {"type": "TEXT", "primary_key": True},
        "text": {"type": "TEXT", "required": True},
        "embedding": {"type": "vector", "dimension": 384}
    }
}

create_table_request = CreateStorageRequest(
    storage_type=StorageType.DATABASE,
    path="test_embeddings",
    data=schema
)

create_table_result = asyncio.run(storage_provider.create_storage_object(create_table_request))
print("Create Table Result:", create_table_result)

## Next Steps

Now that you're familiar with the basics of the Naptha SDK, you can:

1. Explore the interactive SDK Playground to experiment with listed NapthaSDK features (try it [here](https://naptha-sdk-demo.streamlit.app/))
2. Read the comprehensive SDK reference documentation in the [Naptha docs](https://docs.naptha.ai/) to learn about all available functionalities.
3. Join the [Naptha community](https://naptha.ai/naptha-community) to get help, share your projects, and connect with other developers
4. Stay up to date with the latest Naptha news and updates by following us on [Twitter](https://twitter.com/NapthaAI) and [Farcaster](https://warpcast.com/naptha)
